In [23]:
import numpy as np
import pandas as pd
import re
import contractions
import bs4
from nltk.corpus import stopwords
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df = df[['Content', 'Summary', 'Dataset']]

In [4]:
df = df.dropna().reset_index(drop=True).reset_index(drop=True)

In [5]:
df = df.drop_duplicates(subset=['Content']).reset_index(drop=True)

In [6]:
df

,Content,Summary,Dataset
0,New York police are concerned drones could bec...,Police have investigated criminals who have ri...,CNN/Daily Mail
1,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...,CNN/Daily Mail
2,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...,CNN/Daily Mail
3,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...,CNN/Daily Mail
4,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...,CNN/Daily Mail
...,...,...,...
580008,Rabbis Mendel Epstein and Martin Wolmark asked...,Two New Jersey rabbis have been arrested and c...,XSum
580009,The Nigeria Football Federation (NFF) wants to...,Nigeria's players understand they will have to...,XSum
580010,"His discouraging start, which began with two d...",Charlton earned their first victory of the sea...,XSum
580011,The first study of its kind in NI charts theft...,One in five older people in Northern Ireland i...,XSum


In [7]:
soup = bs4.BeautifulSoup

In [8]:
stop_words = stopwords.words("english")

In [9]:
def text_preprocess(text_list):
    
    # Expand contractions
    text_list = [contractions.fix(sentence) for sentence in text_list]
    # Remove HTML tags
    text_list = [soup(sentence, "html.parser").get_text() for sentence in text_list]
    # Keep only alphabet characters and whitespaces
    text_list = [re.sub(r"[^a-zA-Z\s]", " ", sentence).lower() for sentence in text_list]
    # Remove \n
    text_list = [re.sub(r"\n", "", sentence) for sentence in text_list]
    # Remove extra whitespaces
    text_list = [re.sub(" +", " ", sentence) for sentence in text_list]
    # Remove stop words and short words
    text_list = [[word for word in sentence.split() if not word in stop_words and len(word)>=3] 
                 for sentence in text_list]
    text_list = [" ".join(sentence) for sentence in text_list]
    return text_list

In [10]:
st = time.time()

ls = text_preprocess(df['Content'])

et = time.time()
print("Time taken: {:d} h {:d} min {:.2f} s".format(int((et - st)/3600), int(((et - st)%3600)/60), ((et - st)%3600)%60))

Time taken: 0 h 15 min 26.38 s


In [24]:
st = time.time()

s = text_preprocess(df['Summary'])

et = time.time()
print("Time taken: {:d} h {:d} min {:.2f} s".format(int((et - st)/3600), int(((et - st)%3600)/60), ((et - st)%3600)%60))

Time taken: 0 h 1 min 19.35 s


In [26]:
df_clean = pd.DataFrame({'Content':ls, 'Summary':s})
df_clean.tail()

,Content,Summary
580008,rabbis mendel epstein martin wolmark asked hir...,two new jersey rabbis arrested charged plottin...
580009,nigeria football federation nff wants avoid di...,nigeria players understand wait paid outstandi...
580010,discouraging start began two defeats draw conc...,charlton earned first victory season first new...
580011,first study kind charts theft fraud misuse mon...,one five older people northern ireland affecte...
580012,day year old man miraculously survived suicide...,rescuers niagara falls still found body man ju...
